# Scraping reviews from Reclame Aqui website

## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
#import matplotlib.pyplot as plt
#%matplotlib inline


## Create driver

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

## Create Url

In [3]:
def create_url(company, page):
    return f'https://www.reclameaqui.com.br/empresa/{company}/lista-reclamacoes/?pagina={page}'

In [30]:
company = 'upnid'
page = 1
url = create_url(company, page)
url

'https://www.reclameaqui.com.br/empresa/upnid/lista-reclamacoes/?pagina=1'

## Get complaints

In [106]:
def get_html(driver, url):
    begin = time.time()
    driver.get(url)
    time.sleep(1)
    end = time.time()
    if True :#end - begin >= 1:
        driver.refresh()
        script = ("return document.getElements"
        "ByTagName('html')[0].innerHTML")
        html = driver.execute_script(script)
        time.sleep(1)
    return html

In [8]:
#def get_complaint_titles(html):
#    title_list = []
#    soup = BeautifulSoup(html, "html.parser")
#    titles = soup.find_all('p', class_="text-title")
#    for i in range(len(titles)):
#        title_list.append(titles[i]['title'])
#        
#    return title_list

In [85]:
def get_complaints(html):
    soup = BeautifulSoup(html, "html.parser")
    complaints = soup.find_all(class_="link-complain-id-complains")
    return complaints

In [53]:
def get_titles(complaints_chunk):
    title_list = []
    for i in range(len(complaints_chunk)):
        title_list.append(complaints_chunk[i].text.strip())
    return title_list

In [90]:
def get_links(complaints):
    link_list = []
    start_url = 'https://www.reclameaqui.com.br'
    for i in range(len(complaints)):
        link = (start_url + complaints[i].get('href'))
        link_list.append(link)
    return link_list

In [10]:
def get_complaint_content(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all('p', class_="ng-binding")[9].contents[0]

In [101]:
def get_titles_and_links(company, num_pages, driver):
    titles = []
    links = []
    for i in range(1, num_pages+1):
        page_url = create_url(company, i)
        page_html = get_html(driver, page_url)
        page_complaints = get_complaints(page_html)
        page_titles = get_titles(page_complaints)
        page_links = get_links(page_complaints)
        
        titles += page_titles
        links += page_links
        
    return titles, links

In [107]:
get_titles_and_links('upnid',2,driver)

([], [])

In [31]:
html = get_html(driver,url)

In [86]:
complaints = get_complaints(html)

In [87]:
type(complaints_chunk[0])

bs4.element.Tag

In [88]:
titles_1 = get_titles(complaints)
titles_1

['compra TV global',
 'faggio store',
 'compra não recebida',
 'robô brinquedo nao chegou.',
 'Cobranças indevidas',
 'Compra sem esposta',
 'Produtos nao entrgues',
 'Quero a devolução do valor pago',
 'Não recebi meu aparelho',
 'Não recebi meu pedido']

In [89]:
links_1 = get_links(complaints)
links_1

['https://www.reclameaqui.com.br/upnid/compra-tv-global_6fe6i3TBUozMqFED/',
 'https://www.reclameaqui.com.br/upnid/faggio-store_L-T5i0Nd-r7rgJay/',
 'https://www.reclameaqui.com.br/upnid/compra-nao-recebida_CpzaKA0xzMAU1dIk/',
 'https://www.reclameaqui.com.br/upnid/robo-brinquedo-nao-chegou_WrHtEhJA5xNQOzZ_/',
 'https://www.reclameaqui.com.br/upnid/cobrancas-indevidas_0v3N1IELY8JAhuZH/',
 'https://www.reclameaqui.com.br/upnid/compra-sem-esposta_Jhm38oLaer15CJNY/',
 'https://www.reclameaqui.com.br/upnid/produtos-nao-entrgues_BDWAiVl2OfeNd0b-/',
 'https://www.reclameaqui.com.br/upnid/quero-a-devolucao-do-valor-pago_JXJSxB1fy7iEbaUL/',
 'https://www.reclameaqui.com.br/upnid/nao-recebi-meu-aparelho_ZZNM-s5d30WHIfPS/',
 'https://www.reclameaqui.com.br/upnid/nao-recebi-meu-pedido_dCStTq3mwSLhXz6p/']

In [81]:
def get_content(links):
    content_list = []
    for i in range(len(links)):
        url = links_1[i]
        html = get_html(driver,url)
        print(f'Retrieved {i+1} of {len(links)}')
        content = get_complaint_content(html)
        content_list.append(content)
    return content_list

In [82]:
get_content(links_1)

Retrieved 1 of 10
Retrieved 2 of 10
Retrieved 3 of 10
Retrieved 4 of 10
Retrieved 5 of 10
Retrieved 6 of 10
Retrieved 7 of 10
Retrieved 8 of 10
Retrieved 9 of 10
Retrieved 10 of 10


['Fiz duas compras na tv global no dia 14/03/2021 com dois cartões de crédito, uma no Valor de 149.00 e outra no valor de 240,00, esta empresa foi quem recebeu o pagamento, peço providencias para resolver o problema ou fazer o reembolso dos.meus cartões de crédito',
 'fiz uma compra no dia 28/02 . pela empresa Faggio store. a mesma não entregou o produto. já tentei vários contatos através do imail e o sac. sem sucesso não me deram nenhum retorno. estou me sentido lezada. gostaria que  extornassem o meu cartao ou me reembolsassem o valor da compra 140.50. pois no site diz que a compra é segura pela unip .',
 'comprei um produto ja faz  tempo e não a recebi se que mim enviaram o numero  de rastreamento ',
 'Em novembro de 2020 comprei um brinquedo robô no site PG UPNIDSUPERONE foi debitado o valor e nao recebi o msm. solicito estorno do valor pago.',
 'Tem várias cobranças indevidas no meu cartão de crédito, não reconheço essas compras',
 'Então, comprei um produto aí e fiz em dois pagam

In [33]:
html_complaint = get_html(driver,url_complaint)

In [34]:
get_complaint_title(html_complaint)

'Produto não entregue'

In [35]:
get_complaint_content(html_complaint)

'Comprei um processador de alimentos na promoção COMPRE 1 LEVE 2 em 24/02 no valor de R$ 54,80. Até o momento não recebi o produto e nem código de rastreamento.Aguardo uma solução.'